# Crawling by imageUrl

In [4]:
import pandas as pd # pip install pandas
import urllib # pip install urllib
META_PATH = '/home/shaun/sneakers/_data/val_sneakers_df_211206.pkl'
df = pd.read_pickle(META_PATH)
import os
IMAGE_FOLDER_PATH = 'images'
if not os.path.exists(IMAGE_FOLDER_PATH):
    os.mkdir(IMAGE_FOLDER_PATH)
for i, (pid,url) in df[['pid','imageUrl']].iterrows():
    urllib.request.urlretrieve(url, IMAGE_FOLDER_PATH+"/{}.jpg".format(pid))
    
#     break

# RAW Crawling source (Using Selenium, Chromdriver)

In [3]:
from selenium import webdriver
import pandas as pd, json, requests
from selenium.webdriver.common.action_chains import ActionChains
from random import uniform
import time
from IPython.display import display, clear_output
from webdriver_manager.Chrome import ChromeDriverManager
def remove_whereareyou_popup(driver):
    try:
        elm = driver.find_element_by_css_selector( "div [class*='WhereAreYouModal__Container']" )
        elm.find_element_by_tag_name('img').click()
    except selenium.common.exceptions.NoSuchElementException:
        print("no popup")
    return
def update_headers(driver,session):
    headers = {
    "User-Agent":
        "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"
    }
    session.headers.update(headers)
    for cookie in driver.get_cookies():
        c = {cookie['name']: cookie['value']}
        session.cookies.update(c)
    return session
def remove_captcha(driver):
    def foo():
        try:
            obj = driver.find_element_by_css_selector( "div [role*='main']")
            ActionChains(driver).click_and_hold(obj).perform()
            time.sleep(3)
            ActionChains(driver).release(obj).perform()
            time.sleep(4)
        except selenium.common.exceptions.NoSuchElementException:
            print("[remove_captcha]:noSuchElementException")
            pass
    prev_size = driver.get_window_size()
    prev_w,prev_h = prev_size['width'],prev_size['height']
    driver.set_window_size(730, 1380)
    time.sleep(1)
    foo()
    driver.set_window_size(int(prev_w), int(prev_h))
    
    return 

def reset_session(driver=None,session=None):
    if driver:
        driver.close()
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get('https://stockx.com')
    time.sleep(3)
    remove_captcha(driver)
    time.sleep(2)
    remove_whereareyou_popup(driver)
#     time.sleep(2)
#     log_in(driver)
    time.sleep(2)
    remove_captcha(driver)
    session = requests.session()
    session = update_headers(driver,session)
    return driver,session


def get_meta_info(tile):
    elms = tile.find_elements_by_tag_name('div')
    name = elms[3].text
    price_type = elms[6].text
    price = elms[7].text
    release_date = elms[8].text 
    main_url = tile.find_element_by_tag_name('a').get_attribute('href')
    
    return name,price_type,price,release_date,main_url
def load_general_info_detail(driver):
    for script in driver.find_elements_by_tag_name('script'):
        script = script.get_attribute('innerHTML').strip()
        if 'window.preLoaded = ' in script:
            data_json = json.loads(script[len('window.preLoaded = '):-1])
    return data_json
def get_pid(data_json):
    return data_json['product']['uuid']
def load_daliy_avg_bids(pid,session):
    chart_url_base = 'https://stockx.com/api/products/{pid}/chart?start_date={start_date}&end_date=2020-07-31&intervals={num_days}&format=highstock&currency=USD&country=KR'
    # find first bid
    bid_data = session.get(chart_url_base.format(pid=pid,num_days=1,start_date='all')).json()
    if bid_data!=None:
        first_bid_day = bid_data['series'][0]['data'][0][0]
        timedelta = (datetime.datetime.fromtimestamp(1596174754000/1e3)-datetime.datetime.fromtimestamp(first_bid_day/1e3)).days
        data = session.get(chart_url_base.format(pid=pid,num_days=timedelta,start_date='all'))
        data_list = data.json()['series'][0]['data']
    else:
        data_list = []
    return data_list
def get_detail_info(main_url,driver,session):
    driver.get(main_url)
    time.sleep(2)
    data_json = load_general_info_detail(driver)
    pid = get_pid(data_json)
    bid_list = load_daliy_avg_bids(pid,session)
    return data_json,pid,bid_list
def check_captcha(driver):
    try:
        obj = driver.find_element_by_css_selector( "div [role*='main']")
    except selenium.common.exceptions.NoSuchElementException:
        return 
    raise
    return 


"""
If you don't have Chrome, and chrome driver, please install it. 
1)
wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | sudo apt-key add -
2)
sudo sh -c 'echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google.list'
3)
sudo apt-get update
4) 
sudo apt-get install google-chrome-stable
5) 
sudo rm -rf /etc/apt/sources.list.d/google.list
6)
google-chrome --version
7)
pip install webdriver_manager
8) 

"""
import pandas as pd # pip install pandas
import urllib # pip install urllib
META_PATH = '/home/shaun/sneakers/_data/val_sneakers_df_211206.pkl'
df = pd.read_pickle(META_PATH)
import os
IMAGE_FOLDER_PATH = 'images'
if not os.path.exists(IMAGE_FOLDER_PATH):
    os.mkdir(IMAGE_FOLDER_PATH)
    
driver,session = reset_session()
for idx, (main_url) in df[['main_url']].iterrows():
    data_json,pid,bid_list = get_detail_info(main_url=main_url,driver=driver,session=session)
    url = data_json['product']['media']['imageUrl']
    urllib.request.urlretrieve(url, IMAGE_FOLDER_PATH+"/{}.jpg".format(pid))

'\n\n1)\nwget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | sudo apt-key add -\n2)\nsudo sh -c \'echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google.list\'\n3)\nsudo apt-get update\n4) \nsudo apt-get install google-chrome-stable\n5) \nsudo rm -rf /etc/apt/sources.list.d/google.list\n6)\ngoogle-chrome --version\n7)\npip install webdriver_manager\n8) \n\n'